# NewsBuddy NLP 초급 미션 문제지

NewsBuddy는 사용자에게 맞춤형 뉴스 헤드라인 생성, 기사 감성 분석, 그리고 간단한 Q&A 챗봇 기능을 제공하는 서비스입니다.  
다음 세 가지 미션을 해결하여 프로토타입을 완성하세요!

## 미션 1: 키워드 기반 뉴스 헤드라인 생성  
- **목표**: 주어진 긴 문장을 줄여서 요약해서 그럴듯한 뉴스 헤드라인을 생성하는 함수 작성  
- **모델**: Hugging Face의 `t5-small` (영어)  
- **핵심 요구사항**:  
  1. `generate_headline(keyword: str, max_length: int) → str` 함수 구현  
  2. 입력 전처리, 토크나이징, `model.generate()` 호출, 디코딩까지 포함  
  3. 최소 1가지 예시 출력

In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# 모델과 토크나이저 로드
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model     = T5ForConditionalGeneration.from_pretrained("t5-small")

def generate_headline(keyword: str, max_length: int = 30) -> str:
    # 1-1) T5가 이해할 수 있는 프롬프트 형식으로 keyword 래핑
    input_text = f"summarize: {keyword}"

    # 2) 토크나이징
    inputs = tokenizer(input_text, return_tensors="pt")

    # 3) 헤드라인 생성
    outputs = model.generate(
        inputs["input_ids"],
        max_length=max_length,
        num_beams=4,
        early_stopping=True
    )

    # 4) 디코딩
    headline = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return headline

# 예시 실행
print(generate_headline("global warming"))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

global warming is a global warming phenomenon. global warming is a global warming phenomenon.


## 미션 2: BART 기반 간단 챗봇 파인튜닝  
- **목표**: BART를 이용해 질의–응답 챗봇을 파인튜닝하고, 한두 가지 예시 질문에 답변하도록 하기  
- **모델**: Hugging Face의 `facebook/bart-base`  
- **데이터**:  
  - 간단한 질문–응답 쌍을 직접 정의 (`qna_pairs = [{"question": "...", "answer": "..."}]`)  
- **핵심 요구사항**:  
  1. `Dataset` 생성 및 토크나이징  
  2. `Trainer`로 파인튜닝  
  3. `chatbot_respond(query: str) → str` 함수 구현

In [4]:
from datasets import Dataset
import torch
from transformers import (
    BartTokenizer, BartForConditionalGeneration,
    Trainer, TrainingArguments
)

# GPU 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# 1) Q&A 데이터 정의
qna_pairs = [
    {"question": "안녕하세요?", "answer": "반갑습니다! 무엇을 도와드릴까요?"},
    {"question": "오늘 날씨 어때?", "answer": "죄송해요, 현재 API 연동이 안 돼서 알 수 없습니다."},
    {"question": "이름이 뭐야?", "answer": "저는 BART 기반 챗봇이에요."},
    {"question": "몇 살이야?", "answer": "AI는 나이를 먹지 않아요!"},
    {"question": "무슨 일을 할 수 있어?", "answer": "간단한 질문에 답할 수 있어요."}
]
ds = Dataset.from_list(qna_pairs)

# 2) 토크나이징
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

def preprocess(batch):
    inputs = [f"question: {q}" for q in batch["question"]]
    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=64)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(batch["answer"], padding="max_length", truncation=True, max_length=64)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

ds = ds.map(preprocess, batched=True)

# 3) 모델과 Trainer 설정
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
training_args = TrainingArguments(
    output_dir="./bart-chatbot",
    per_device_train_batch_size=4,
    num_train_epochs=100,
    logging_steps=10,
    save_strategy="no",
    report_to="none",  # wandb 미사용
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds
)

# 4) 파인튜닝
trainer.train()

# 5) 챗봇 응답 함수
def chatbot_respond(query: str, max_length: int = 50) -> str:
    input_text = f"question: {query}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=64)

    # ✅ 디바이스 통일
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    outputs = model.generate(
        inputs["input_ids"],
        max_length=max_length,
        num_beams=4,
        early_stopping=True
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Step,Training Loss
10,6.476000
20,4.731100
30,3.309000
40,2.636200
50,2.032200
60,1.336500
70,1.114800
80,0.612700
90,0.514900
100,0.381900


반갑습니다! 무엇을 도와드릴까요?
저는 BART 기반 챗봇이에요.


In [9]:
# 🔍 예시 실행
print(chatbot_respond("안녕하세요?"))
print(chatbot_respond("이름이 뭐야?"))
print(chatbot_respond("오늘 날씨 어때?"))

반갑습니다! 무엇을 도와드릴까요?
저는 BART 기반 챗봇이에요.
죄송해요, 현재 API 연동이 알 수 없습니다.
